In [1]:
import numpy as np 
import open3d as o3d
import cv2
import pandas as pd
import glob

import torch
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import sys

In [2]:
def to_3D(fx, fy, depth, cx, cy):
    H, W = depth.shape

    u, v = np.meshgrid(np.arange(W), np.arange(H))

    x = (u - cx) * depth / fx
    y = (v - cy) * depth / fy
    z = depth

    return np.stack((x, y, z), axis=-1)   # HxWx3


def make_o3d_pointcloud(rgb_path, depth_path, fx, fy, cx, cy):
    
    if type(rgb_path) != str:
        rgb = rgb_path
    else:
        rgb = cv2.imread(rgb_path)[:, :, ::-1]
    
    if type(depth_path) != str:
        depth = depth_path
    else:
        depth = np.load(depth_path).astype(np.float32) / 1000 ## convert mm depth map to meter

    # ----- CREATE MASK -----
    mask = (depth > 0) & np.isfinite(depth)   # boolean mask HxW

    # Unproject entire depth → xyz
    xyz = to_3D(fx, fy, depth, cx, cy)   # HxW x 3

    # Flatten arrays
    xyz_flat = xyz.reshape(-1, 3)
    rgb_flat = rgb.reshape(-1, 3)
    mask_flat = mask.reshape(-1)

    # Apply mask
    xyz_valid = xyz_flat[mask_flat]
    rgb_valid = rgb_flat[mask_flat] / 255.0

    # Build point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz_valid)
    pcd.colors = o3d.utility.Vector3dVector(rgb_valid)

    return pcd


In [4]:
project_folder = "./data/datasets/final_project_dataset"
rgb_dir = f"{project_folder}/crazyflie-rgb-images"
pose_dir = f"{project_folder}/crazyflie-poses"

fx = 218.792814876677
fy = 224.37416225752526
cx = 322.66660990415625
cy = 243.22672826370496

In [5]:

rgb_files = sorted(glob.glob(f"{rgb_dir}/*.jpg"))
pose_files = sorted(glob.glob(f"{pose_dir}/*.txt"))
df = pd.DataFrame({
    "rgb_path": rgb_files,     # stores the actual path
    "pose_path": pose_files
})
df.head()

,rgb_path,pose_path
0,./data/datasets/final_project_dataset/crazyfli...,./data/datasets/final_project_dataset/crazyfli...
1,./data/datasets/final_project_dataset/crazyfli...,./data/datasets/final_project_dataset/crazyfli...
2,./data/datasets/final_project_dataset/crazyfli...,./data/datasets/final_project_dataset/crazyfli...
3,./data/datasets/final_project_dataset/crazyfli...,./data/datasets/final_project_dataset/crazyfli...
4,./data/datasets/final_project_dataset/crazyfli...,./data/datasets/final_project_dataset/crazyfli...


In [6]:
# You can visualize the rgb images here:

def show_img(path):
    return f"<img src='{path}' width='300'/>"

styled = df.head(2).style.format({
    "rgb_path": show_img     # render image instead of path
}).hide(axis="index")        # optional: hide row numbers

display(styled)

rgb_path,pose_path
,./data/datasets/final_project_dataset/crazyflie-poses/crazyflie_frame-001600.pose.txt
,./data/datasets/final_project_dataset/crazyflie-poses/crazyflie_frame-001601.pose.txt


## Set up model

### Zoe Depth setup

In [ ]:

sys.path.insert(0, "models/ZoeDepth")
from zoedepth.models.builder import build_model
from zoedepth.utils.config import get_config
from zoedepth.utils.misc import get_image_from_url, colorize


# Device
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Load model
conf = get_config("zoedepth", "eval")
model_zoe = build_model(conf)
zoe = model_zoe.to(DEVICE)
zoe.eval()

img_size [384, 512]


Using cache found in /Users/manoruo/.cache/torch/hub/intel-isl_MiDaS_master
/opt/homebrew/anaconda3/envs/RL/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:

img = Image.open("/home/mikea/Documents/Projects/Crazyflie/monocular_reconstruction/MonoNav/data/demo_hallway/crazyflie-rgb-images/crazyflie_frame-000001.rgb.jpg")
# Run ZoeDepth
depth_numpy = zoe.infer_pil(img)

# Colorize depth for visualization
colored_depth = colorize(depth_numpy)

# Plot
fig, axs = plt.subplots(1, 2, figsize=(15, 7))
for ax, im, title in zip(axs, [img, colored_depth], ['Input', 'Predicted Depth']):
    ax.imshow(im)
    ax.axis('off')
    ax.set_title(title)
plt.show()

pcd = make_o3d_pointcloud(np.array(img), depth_numpy, fx, fy, cx, cy)
o3d.visualization.draw_geometries([pcd])
